In [ ]:
# https://inspire.geoportaal.ee/geoserver/SU_ruutkaart/wfs?service=WFS&version=2.0.0&request=GetCapabilities
# https://metadata.geoportaal.ee/geonetwork/srv/est/catalog.search#/metadata/23e616e7-aad2-47b8-bdc8-60b6f56fdf16
# application/json
# SU_ruutkaart:SU.StatisticalGridCell
# count / STARTINDEX=10
# https://inspire.geoportaal.ee/geoserver/SU_ruutkaart/wfs?service=WFS&version=2.0.0&request=GetFeature&version=2.0.0&outputFormat=application/json&typenames=SU_ruutkaart:SU.StatisticalGridCell&count=10

In [11]:
import requests
import json

r = requests.get("https://inspire.geoportaal.ee/geoserver/SU_ruutkaart/wfs",
                 params={'service':'WFS',
                         'request':'GetFeature',
                         'version':'2.0.0',
                         'outputFormat':'application/json',
                         'typenames':'SU_ruutkaart:SU.StatisticalGridCell',
                         'count':50000,
                         'startindex':0} )

In [12]:
r.status_code

200

In [13]:
data = r.json()

In [14]:
len ( data['features'] )
    

45636

In [15]:
data['features'][0]

{'type': 'Feature',
 'id': 'SU.StatisticalGridCell.1kmN3925E5032',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[383452.4784, 6417726.1088],
    [383655.7664, 6418705.4155],
    [384636.0814, 6418508.5624],
    [384432.8007, 6417529.2583],
    [383452.4784, 6417726.1088]]]},
 'geometry_name': 'geom',
 'properties': {'gml_id': '1kmN3925E5032',
  'gml_identifier': '1kmN3925E5032-ee.stat.su-stat-ruudustik_1km',
  'gml_identifier_codespace': 'http://inspire.ec.europa.eu/ids',
  'gml_metadataproperty_xlink_href': 'https://metadata.geoportaal.ee/geonetwork/srv/est/xml.metadata.get?uuid=stat_grid',
  'gml_metadataproperty_xlink_title': 'Eesti statistiline ruutvõrk 1x1km',
  'gml_original_coordinate_system': 'http://www.opengis.net/def/crs/EPSG/0/3035',
  'gml_parent_property': 'featureMember',
  'gml_description': 'Last update: 2021-03-10',
  'code': 'EE_CRS_3035-RES_1000m-N3925000E5032000',
  'grid_xlink_href': 'https://inspire.ec.europa.eu/codelist/VoidReasonValue/Unpopulated',
  'gr

In [16]:
# Serializing json 
json_object = json.dumps(data, indent = 4)
  
# Writing to sample.json
with open("stats.geojson", "w") as outfile:
    outfile.write(json_object)

In [24]:
import geopandas as gpd

sdf = gpd.read_file("stats.geojson", driver="GeoJSON")

In [25]:
sdf84 = sdf.to_crs(4326) # strange that the metadata indocates this data should be in 3035, but geopandas takes it as 3301 and correctly reprojects

In [26]:
sdf84.head(2)

,id,gml_id,gml_identifier,gml_identifier_codespace,gml_metadataproperty_xlink_href,gml_metadataproperty_xlink_title,gml_original_coordinate_system,gml_parent_property,gml_description,code,grid_xlink_href,grid_xlink_title,lowers_xlink_href,lowers_xlink_title,upper_xlink_href,upper_xlink_title,xml_ns_uri,geometry
0,SU.StatisticalGridCell.1kmN3925E5032,1kmN3925E5032,1kmN3925E5032-ee.stat.su-stat-ruudustik_1km,http://inspire.ec.europa.eu/ids,https://metadata.geoportaal.ee/geonetwork/srv/...,Eesti statistiline ruutvõrk 1x1km,http://www.opengis.net/def/crs/EPSG/0/3035,featureMember,Last update: 2021-03-10,EE_CRS_3035-RES_1000m-N3925000E5032000,https://inspire.ec.europa.eu/codelist/VoidReas...,unpopulated,https://inspire.ec.europa.eu/codelist/VoidReas...,Unpopulated,https://inspire.ec.europa.eu/codelist/VoidReas...,Unpopulated,http://inspire.ec.europa.eu/schemas/su-grid/4.0,"POLYGON ((22.03507 57.88583, 22.03802 57.89467..."
1,SU.StatisticalGridCell.1kmN3926E5031,1kmN3926E5031,1kmN3926E5031-ee.stat.su-stat-ruudustik_1km,http://inspire.ec.europa.eu/ids,https://metadata.geoportaal.ee/geonetwork/srv/...,Eesti statistiline ruutvõrk 1x1km,http://www.opengis.net/def/crs/EPSG/0/3035,featureMember,Last update: 2021-03-10,EE_CRS_3035-RES_1000m-N3926000E5031000,https://inspire.ec.europa.eu/codelist/VoidReas...,unpopulated,https://inspire.ec.europa.eu/codelist/VoidReas...,Unpopulated,https://inspire.ec.europa.eu/codelist/VoidReas...,Unpopulated,http://inspire.ec.europa.eu/schemas/su-grid/4.0,"POLYGON ((22.02140 57.89618, 22.02434 57.90502..."


In [50]:
sdf84 = sdf84[['gml_id', 'geometry']].rename(columns={'gml_id':'name'})

In [31]:
import os

os.remove("stats.geojson")

In [51]:
sdf84.to_file("stats84.json", driver="GeoJSON")

In [59]:
import pydeck as pdk


DATA_URL = "https://allixender.github.io/30daymaps2021/maps/day-03/stats84.json"

INITIAL_VIEW_STATE = pdk.ViewState(latitude=58.37, longitude=26.72, zoom=12, bearing=15, pitch=25,
                                  # min_zoom=6, max_zoom=14
                                  )

geojson = pdk.Layer(
    "GeoJsonLayer",
    DATA_URL,
    opacity=0.1,
    stroked=True,
    pickable=True,
    filled=True,
    # extruded=False,
    get_fill_color="[0, 220, 255]",
    get_line_color="[255, 255, 255]",
    line_width_scale=7,
    line_width_min_pixels=1,
    get_line_width=3,
    get_elevation=10
)


r = pdk.Deck(layers=[geojson], initial_view_state=INITIAL_VIEW_STATE, map_style="dark")

In [60]:
r.to_html("json_layer.html")